## Import Packages

In [ ]:
%matplotlib notebook

import pandas as pd
import numpy as np
from scipy import sparse as sp
import networkx as nx
from sklearn.preprocessing import StandardScaler
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [3]:
title_font = {'size':'18', 'color':'black', 'verticalalignment':'bottom',  'fontstyle':'bold'} 
axis_font = { 'size':'14'}

## Import Dataset

In [ ]:
aut_df = pd.read_csv('/Users/codyotoole/Desktop/Research_Data/GeneDrive_Data/MetaData/Authors_10.csv')
p_df = pd.read_csv('/Users/codyotoole/Desktop/Research_Data/GeneDrive_Data/MetaData/gd_full_meta_20002018_10auth.csv')
ap_df = pd.read_csv('/Users/codyotoole/Desktop/Research_Data/GeneDrive_Data/MetaData/PaperAuthors_10.csv')

### Learn About the Data

In [ ]:
print('Network contain %s papers from %s authors.'%(len(ap_df.PaperID.unique()), 
                                                     len(ap_df.AuthorID.unique())))

In [ ]:
#Number of publications per author
plt.figure(figsize=(6,3))
mpl.rcParams['axes.linewidth'] = 1 #set the value globally
mpl.rcParams['axes.edgecolor'] = 'k'
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False

aut_pap_cnt = dict(ap_df['AuthorID'].value_counts())

xmin = min(aut_pap_cnt.values())
xmax = max(aut_pap_cnt.values())
bins = (xmax-xmin)
plt.xlim(xmin-1, xmax+1)

ax = plt.hist(list(aut_pap_cnt.values()), bins=bins, log=1, width=0.95, color='r', edgecolor='none', alpha=0.8 );
# [i.set_linewidth(0.1) for i in ax.spines.itervalues()]

plt.xlabel('# Papers per author', **axis_font)
plt.ylabel('# Authors ', **axis_font)

plt.grid(0)

In [ ]:
#the kinds of papers each author tends to publish 
ptype_df = p_df[['PaperID', 'PaperType']] 
ptype_df = ptype_df.join(pd.get_dummies(ptype_df['PaperType']), how='outer')
ap_df = ap_df.merge(ptype_df.rename(columns={'ID':'PaperID'}), on='PaperID').drop('ID', axis =1)
nip_df = ap_df.copy()
nip_df.drop('PaperType', axis=1, inplace=True)
nip_df = nip_df.groupby('AuthorID').sum().reset_index()
nip_df = nip_df.merge(aut_df.rename(columns={'Id':'AuthorID'}), on='AuthorID')
auth = nip_df['Author']
nip_df.drop(labels=['Author'], axis=1,inplace = True)
nip_df.insert(0, 'Author', auth)

nip_df['Total_Publications'] = nip_df['GE research']+nip_df['ethics/policy']+nip_df['gene research']+nip_df['insect research']+nip_df['population dynamics']+nip_df['review']+nip_df['wolbachia']          

for i in 'GE research','ethics/policy', 'gene research', 'insect research', 'population dynamics', 'review', 'wolbachia':
    nip_df[i+'_ratio'] = nip_df[i]/nip_df['Total_Publications']

nip_df.sort_values(by=['Total_Publications'], ascending=False, inplace=True)

## Construct Network

In [ ]:
#author-author interaction network
#first make a bipartite graph of author-paper
''' First need to map both author and paper ids to start
    from zero so that we can store them in a numpy array. '''

int_pid = dict(enumerate(list(ap_df.PaperID.unique())))
int_aid = dict(enumerate(list(ap_df.AuthorID.unique())))

#make index of papers and authors
pid_intid = {v:k for k,v in int_pid.items()}
aid_intid = {v:k for k,v in int_aid.items()}


ap_tuples = list(zip(ap_df.AuthorID, ap_df.PaperID))
ap_int_tups =  [(aid_intid[i[0]], pid_intid[i[1]]) for i in ap_tuples]

''' AP: matrix of author-paper, AP[i, j]=1 indicates that author i has published paper j '''
AP = sp.csc_matrix((np.ones(len(ap_int_tups)), zip(*ap_int_tups)))


''' AA: the author-author matrix, 
    AA[i, j]=1 indicates that author i has published a paper with author j '''
AA = AP.dot(AP.T)

In [ ]:
#converting numpy 2D array to network
'Remove self-loops'
AA = np.array(AA - np.diag(AA.diagonal()))

'Weighted graph'
G = nx.from_numpy_matrix(AA, parallel_edges=True)

### Centrality Measures

In [ ]:
#Compute network centrality measures to add to dataframe
'''number of co-authors per member '''
node_deg = nx.degree(G) 

'''normalized number of co-authors '''
deg_cent = nx.degree_centrality(G) 

'''fraction of the number of times the author appears on the path connecting two other authors '''
bet_cent = nx.betweenness_centrality(G)

close_cent = nx.closeness_centrality(G)

eigen_cent = nx.eigenvector_centrality(G)

nip_df['Degree'] = nip_df['AuthorID'].apply(lambda l: node_deg[aid_intid.get(l)])
nip_df['Deg_Cent'] = nip_df['AuthorID'].apply(lambda l: deg_cent[aid_intid.get(l)])
nip_df['Betweenness'] = nip_df['AuthorID'].apply(lambda l: bet_cent.get(aid_intid.get(l)))
nip_df['Closeness'] = nip_df['AuthorID'].apply(lambda l: close_cent.get(aid_intid.get(l)))
nip_df['Eigenvector'] = nip_df['AuthorID'].apply(lambda l: eigen_cent.get(aid_intid.get(l)))

### Network Visualization

In [ ]:
print(G.number_of_edges(), G.number_of_nodes(), nx.density(G))

In [ ]:
K = G.copy()

In [ ]:
aid_name = dict(zip(aut_df.Id, aut_df.Author))

for node in K.nodes():
    name = aid_name.get(int_aid[node])

In [ ]:
nid_intid = {}
    
for j,c in aid_intid.items():
    for k,v in aid_name.items():
        if k == j:
            nid_intid[c] = v
            
K=nx.relabel_nodes(K,nid_intid)
#Relabels nodes with real author names

In [ ]:
#In my network I want to focus on the giant component
#So I need to remove the isolates

nip_sub = nip_df[(nip_df['GE research']+nip_df['ethics/policy']+nip_df['gene research']+nip_df['insect research']+nip_df['population dynamics']+nip_df['review']+nip_df['wolbachia'])>1]
sub_authors =nip_sub['Author'].unique().tolist()

In [ ]:
H = K.copy()

In [ ]:
for component in list(nx.connected_components(H)):
    if any(x in sub_authors for x in list(component)):
        continue
    else:
            for node in component:
                        H.remove_node(node)
 
len(list(H.nodes))

#### Convert to Gephi for Visualization!

In [ ]:
nx.write_gexf(H, 'gene_drive.gexf')

## Network Analysis
Average shortest path, degree correlation, degree distribution, rich club, preferential attachment, strength of ties

### Average Path Length

In [ ]:
nx.average_shortest_path_length(H)

### Degree Correlation

In [ ]:
nx.degree_pearson_correlation_coefficient(H)
#Social Networks are often assortative due to formation of communities

### Degree Distribution

In [ ]:
degrees = G.degree()
degree_values = dict(degrees).values()
degree_values = sorted(degree_values)
histogram = [degree_values.count(i)/float(nx.number_of_nodes(G)) \
for i in degree_values]

import matplotlib.pyplot as plt

plt.plot(degree_values,histogram, 'o', color='r', markersize = 5)
plt.xlabel('k')
plt.ylabel('p(k)')
plt.xscale('log')
plt.yscale('log')

### Preferential Attachment

In [ ]:
degrees = G.degree()
degree_values = dict(degrees).values()
degree_values = sorted(degree_values)
k = np.asarray(degree_values)
j = np.asarray(degree_values)

pa = []
for n in k:
    p = n/(np.sum(j))
    pa.append((n, p))
 
list1, list2 = zip(*pa) 


def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list

list2 = unique(list2)

pa3 = []
for n in range(0, len(list2)):
    p = np.sum(list2[0:n+1])
    pa3.append(p)
            
deg = list(set(degree_values))
import matplotlib.pyplot as plt

plt.plot(deg, pa3, 'o', mfc='none', color='r', markersize=10, markeredgewidth=2.2)
#plt.plot(deg, [number ** 2 for number in deg], '-')
#plt.plot(deg, deg, '-')
plt.xlabel('k')
plt.ylabel('p(k)')
plt.xscale('log')
plt.yscale('log')
plt.xlim(0.8, 10**2)
plt.ylim(10**-4.2, 1)

### Rich-club Structure (The rich get richer)

In [ ]:
from networkx.utils import accumulate
from networkx.utils import not_implemented_for

def _compute_rc(G):
    """Returns the rich-club coefficient for each degree in the graph
    `G`.

    `G` is an undirected graph without multiedges.

    Returns a dictionary mapping degree to rich-club coefficient for
    that degree.

    """
    deghist = nx.degree_histogram(G)
    total = sum(deghist)
    # Compute the number of nodes with degree greater than `k`, for each
    # degree `k` (omitting the last entry, which is zero).
    nks = (total - cs for cs in accumulate(deghist) if total - cs > 1)
    # Create a sorted list of pairs of edge endpoint degrees.
    #
    # The list is sorted in reverse order so that we can pop from the
    # right side of the list later, instead of popping from the left
    # side of the list, which would have a linear time cost.
    edge_degrees = sorted((sorted(map(G.degree,  e)) for e in G.edges()),
                          reverse=True)
    ek = G.number_of_edges()
    k1, k2 = edge_degrees.pop()
    rc = {}
    for d, nk in enumerate(nks):
        while k1 <= d:
            if len(edge_degrees) == 0:
                ek = 0
                break
            k1, k2 = edge_degrees.pop()
            ek -= 1
        rc[d] = 2 * ek / (nk * (nk - 1))
    return rc


# make R a copy of G, randomize with Q*|E| double edge swaps
# and use rich_club coefficient of R to normalize
rr = {}
rc = _compute_rc(G)
R = G.copy()
E = R.number_of_edges()
nx.double_edge_swap(R, 100 * E, max_tries=100 * E * 10, seed=None)
rcran = _compute_rc(R)

for k, v in rc.items():
                rr[k] = v / rcran[k]
            

keys = [] 
values = [] 
items = rr.items() 
for item in items: 
    keys.append(item[0]), values.append(item[1])


plt.plot(keys, values, 'o', mfc='none', color='r', markersize=10, markeredgewidth=2.2)
#plt.plot(deg, [number ** 2 for number in deg], '-')
plt.axhline(y=1, color='black', linestyle='-')
plt.xlabel('k')
plt.ylabel('p(k)')
plt.xscale('log')
plt.yscale('log')

### Strength of Ties

In [ ]:
weight = []
for u,v,data in H.edges(data=True):
    weight.append((u, v, data['weight']))
    
weight_df = pd.DataFrame(weight, columns=["Author1", "Author2", "Edge_Weight"])
    
qqq = []
for n in range(0, len(weight_df)):
    if weight_df['Edge_Weight'][n] > 2:
        qqq.append(weight_df['Edge_Weight'][n])

print(len(qqq))

## Final Visualization (Community Detection through Node2Vec)

In [ ]:
from node2vec import Node2Vec

In [ ]:
# Generate walks
node2vec = Node2Vec(H, dimensions=10, walk_length=10, num_walks=100)

# Learn embeddings 
model = node2vec.fit(window=10, min_count=1)

In [ ]:
#Get vectors into numpy array
X3 = np.array(model.wv.vectors, dtype='float') 

node_sub = list(H.nodes)

X3_vectors = []

for node in node_sub:
    X3_vectors.append(model[node])
    
X3_vectors = np.array(X3_vectors)

pca = PCA(n_components=0.95, random_state=42)
X3_reduced= pca.fit_transform(X3_vectors)
X3_reduced.shape

In [ ]:
# run kmeans with many different k

distortions = []
K = range(2, 30)
for k in K:
    k_means = KMeans(n_clusters=k, random_state=42).fit(X3_reduced)
    k_means.fit(X3_reduced)
    #x1.append(k)
    distortions.append(sum(np.min(cdist(X3_reduced, k_means.cluster_centers_, 'euclidean'), axis=1)) / X3_vectors.shape[0])
    #print('Found distortion for {} clusters'.format(k))

In [ ]:
#Elbow method
X_line = [K[0], K[-1]]
Y_line = [distortions[0], distortions[-1]]

# Plot the elbow
plt.plot(K, distortions, 'b-')
plt.plot(X_line, Y_line, 'r')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
#Specify colors and clusters you like
n_cluster = 6
kmeans = KMeans(n_clusters=n_cluster, random_state=42)
y_pred = kmeans.fit_predict(X3_reduced)

kmeans_df = pd.DataFrame({'Author':node_sub})
kmeans_df['cluster'] = kmeans.labels_
kmeans_df['color'] = kmeans_df['cluster']


for n in range(0, len(kmeans_df['cluster'])):
    if kmeans_df['cluster'][n] == 0:
        kmeans_df['color'][n] = 'green'
    if kmeans_df['cluster'][n] == 1:
        kmeans_df['color'][n] = 'blue'
    if kmeans_df['cluster'][n] == 2:
        kmeans_df['color'][n] = 'yellow'
    if kmeans_df['cluster'][n] == 3:
        kmeans_df['color'][n] = 'red'
    if kmeans_df['cluster'][n] == 4:
        kmeans_df['color'][n] = 'purple'
    if kmeans_df['cluster'][n] == 5:
        kmeans_df['color'][n] = 'orange'
    if kmeans_df['cluster'][n] == 6:
        kmeans_df['color'][n] = 'violet'

In [ ]:
for n in range(0, len(kmeans_df)):  
    H.nodes[kmeans_df['Author'][n]]['color'] = kmeans_df['color'][n]

In [ ]:
#Export for gephi viz again
nx.write_gexf(H, 'node2vec_gd.gexf')